In [1]:
from src.data_utils.dataset import build_datset_train_val
from src.rnn.gru_prototipes import build_gru_model
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf

2023-08-18 01:57:19.465017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 01:57:20.106855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-18 01:57:20.820524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 01:57:20.839883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=256)

train split: 28160 | val split: 6656


In [3]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=get_predefine_callbacks(model_name="v1"))
    print(model.summary())

    val_loss = model.evaluate(val_dataset_one_hot)[0]
    return  val_loss

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-18 01:57:21,958] A new study created in memory with name: no-name-941b4981-1783-4bad-950a-dc6ae1efcb16


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/20


2023-08-18 01:57:40.015189: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT8
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_41/output/_22'
2023-08-18 01:57:40.253623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-08-18 01:57:40.353481: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3155bfb090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-18 01:5

     69/Unknown - 137s 2s/step - loss: 3.4850 - accuracy: 0.1358